In [1]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git
%pip  install -r FastSAM/requirements.txt
# install CLIP
%pip  install git+https://github.com/openai/CLIP.git
# install SAM
%pip  install git+https://github.com/facebookresearch/segment-anything.git
# install other dependencies
%pip  install roboflow supervision jupyter_bbox_widget
%pip install roboflow

Cloning into 'FastSAM'...
remote: Enumerating objects: 1171, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 1171 (delta 201), reused 186 (delta 186), pack-reused 929
Receiving objects: 100% (1171/1171), 72.45 MiB | 40.12 MiB/s, done.
Resolving deltas: 100% (456/456), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 468.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="OZmt7veKofDCRjgyQ5XE")
project = rf.workspace("csilab").project("csilab-braintumor")
dataset = project.version(3).download("yolov8")

import os
HOME = os.getcwd()
print("HOME:", HOME)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.120, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to CsiLab-BrainTumor-3 in yolov8:: 100%|██████████| 618/618 [00:00<00:00, 6390.45it/s]

HOME: /content


In [3]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt


In [4]:
FAST_SAM_CHECKPOINT_PATH = "/content/weights/FastSAM.pt"

In [5]:
%cd {HOME}/FastSAM

import os
import cv2
import torch
import roboflow
import base64

import supervision as sv
import numpy as np

from roboflow import Roboflow
from fastsam import FastSAM, FastSAMPrompt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

/content/FastSAM


In [6]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

fast_sam = FastSAM(FAST_SAM_CHECKPOINT_PATH)

In [7]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask_np.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    ob_id+=1


  return labels

In [8]:
import os
path = dataset.location + "/train/images/"
Output_Path = dataset.location +"/output/"

if not os.path.exists(Output_Path):
    os.makedirs(Output_Path)
    print("Criado diretorio: ", Output_Path)

for IMAGE in os.listdir(path):
   if (IMAGE.endswith(".jpg")):
      IMAGE_PATH = path+"/"+IMAGE
      results = fast_sam(
        source=IMAGE_PATH,
        device=DEVICE,
        retina_masks=True,
        imgsz=1024,
        conf=0.5,
        iou=0.5)
      prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
      masks = prompt_process.everything_prompt()
      prompt_process.plot(annotations=masks, output_path = Output_Path + IMAGE)
      #torch.save(masks, (Output_Path + IMAGE).split(".")[0] + ".pt")
      yolo_labels = convert_masks_to_yolo_labels(masks)

      '''out_file_path = (Output_Path + IMAGE).split(".jpg")[0] + ".txt"
      with open(out_file_path, "w") as out_file:
          for label in yolo_labels:
              out_file.write(" ".join(map(str, label)) + "\n")'''
      #separando por objetos detectatos
      for label in yolo_labels:
        out_file_path = (Output_Path + IMAGE).split(".jpg")[0]+ "object_" + str(label[0]) + ".txt"
        out_file = open(out_file_path, "w")
        out_file.write(" ".join(map(str, label)) + "\n")



Criado diretorio:  /content/CsiLab-BrainTumor-3/output/



image 1/1 /content/CsiLab-BrainTumor-3/train/images/Te-me_0080_jpg.rf.21244eb59094d52820768b3571a986fb.jpg: 1024x1024 3 objects, 161.9ms
Speed: 11.9ms preprocess, 161.9ms inference, 57.2ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/43_jpg.rf.5efa8f4265063724a49d949143493002.jpg: 1024x1024 2 objects, 165.8ms
Speed: 5.8ms preprocess, 165.8ms inference, 6.0ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/80_jpg.rf.cad74fa003f2c04d77984ad010117bc2.jpg: 1024x1024 2 objects, 168.6ms
Speed: 5.6ms preprocess, 168.6ms inference, 4.5ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/28_jpg.rf.d7ac3d185bcd4ea8d181ed751d403207.jpg: 1024x1024 3 objects, 164.4ms
Speed: 7.3ms preprocess, 164.4ms inference, 3.2ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/3_jpg.rf.c3c830f9

No object detected.


image 1/1 /content/CsiLab-BrainTumor-3/train/images/Te-me_0084_jpg.rf.686a065edd0966b37721df9403d42996.jpg: 1024x896 5 objects, 166.9ms
Speed: 4.0ms preprocess, 166.9ms inference, 3.2ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/56_jpg.rf.5d7e23942786036511bf1539b8cddce9.jpg: 1024x1024 4 objects, 178.3ms
Speed: 4.5ms preprocess, 178.3ms inference, 3.3ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/Te-me_0027_jpg.rf.cd75b947bb66739fc12c33355800613d.jpg: 1024x960 2 objects, 175.8ms
Speed: 4.5ms preprocess, 175.8ms inference, 3.9ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/Te-me_0083_jpg.rf.9789fded79e28d7db76e4e6390fa938e.jpg: 992x1024 1 object, 175.3ms
Speed: 7.4ms preprocess, 175.3ms inference, 3.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-3/train/images/52_jpg.r

In [9]:
yolo_labels = convert_masks_to_yolo_labels(masks)
for label in yolo_labels:
  print(label[0])



#model = torch.load('/content/CsiLab-BrainTumor-3/output/45_jpg.pt')


0
1
2


In [14]:
IMAGE_PATH = os.path.join(dataset.location, 'train', 'images/45_jpg.rf.36ef270424c1c2114e972a731368c5df.jpg')

In [15]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=False,
    imgsz=1024,
    conf=0.5,
    iou=0.5)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.everything_prompt()
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")

yolo_labels = convert_masks_to_yolo_labels(masks)
print(yolo_labels)



image 1/1 /content/CsiLab-BrainTumor-3/train/images/45_jpg.rf.36ef270424c1c2114e972a731368c5df.jpg: 1024x1024 4 objects, 171.2ms
Speed: 6.3ms preprocess, 171.2ms inference, 11.6ms postprocess per image at shape (1, 3, 1024, 1024)


[[0, 0.5087890625, 0.62109375, 0.087890625, 0.1484375], [1, 0.4052734375, 0.599609375, 0.1474609375, 0.0830078125], [2, 0.42578125, 0.4375, 0.21875, 0.2578125], [3, 0.4765625, 0.5, 0.8740234375, 1.0]]


In [19]:
def annotate_image(image_path: str, masks: np.ndarray) -> np.ndarray:
    image = cv2.imread(image_path)

    xyxy = sv.mask_to_xyxy(masks=masks)
    detections = sv.Detections(xyxy=xyxy, mask=masks)

    mask_annotator = sv.MaskAnnotator()
    return mask_annotator.annotate(scene=image.copy(), detections=detections)

In [20]:
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [21]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image(IMAGE_PATH)
widget

BBoxWidget(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#b…

In [26]:

widget.bboxes[0]['label'] = "tumor"
widget.bboxes

[{'x': 79, 'y': 75, 'width': 33, 'height': 38, 'label': 'tumor'}]

In [27]:

box = widget.bboxes[0] if widget.bboxes else default_box
box = [
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
]
print(box)

[79, 75, 112, 113]


In [28]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    #mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    #mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    ob_id+=1


  return labels

In [31]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=True,
    imgsz=1024,
    conf=0.5,
    iou=0.5)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")
masks = prompt_process.box_prompt(bbox=box)

yolo_labels = convert_masks_to_yolo_labels(masks)

for label in yolo_labels:
  out_file_path = f"{HOME}/output_"+ "object_" + str(label[0]) + ".txt"
  out_file = open(out_file_path, "w")
  out_file.write(" ".join(map(str, label)) + "\n")
  print(label)

#converter mask to .txt to yollov8
#criear dataset com imagens instanciadas no roboflow


image 1/1 /content/CsiLab-BrainTumor-3/train/images/45_jpg.rf.36ef270424c1c2114e972a731368c5df.jpg: 1024x1024 4 objects, 178.2ms
Speed: 5.6ms preprocess, 178.2ms inference, 3.3ms postprocess per image at shape (1, 3, 1024, 1024)


[0, 0.42410714285714285, 0.4375, 0.21875, 0.25892857142857145]


In [16]:
def convert_masks_to_yolo_labels(masks, class_ids):
  labels = []
  for mask, class_id in zip(masks, class_ids):
    bounding_box = cv2.boundingRect(mask.astype(np.uint8))
    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [class_id, x_center, y_center, width, height]
    labels.append(label)

  return labels

print(convert_masks_to_yolo_labels(masks, [1])[0])

AttributeError: ignored

In [ ]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=True,
    imgsz=1024,
    conf=0.5,
    iou=0.9)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.everything_prompt()

In [ ]:
masks = masks.cpu().numpy().astype(bool)

annotated_image=annotate_image(image_path=IMAGE_PATH, masks=masks)
sv.plot_image(image=annotated_image, size=(8, 8))

In [ ]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=False,
    imgsz=1024,
    conf=0.5,
    iou=0.9)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.text_prompt(text='lighter brain tumor inside an brain')
prompt_process.plot(annotations=masks, output_path = f"{HOME}/output" + ".jpg")

In [49]:
def calculate_iou(box1, box2):
    # box1 e box2 devem ser no formato [x, y, width, height]

    # Extrair as coordenadas das bounding boxes
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calcular as coordenadas dos cantos superiores esquerdos e inferiores direitos
    x1_min, y1_min, x1_max, y1_max = x1, y1, x1 + w1, y1 + h1
    x2_min, y2_min, x2_max, y2_max = x2, y2, x2 + w2, y2 + h2

    # Calcular a área da interseção
    x_intersection = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
    y_intersection = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
    intersection_area = x_intersection * y_intersection

    # Calcular a área da união
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - intersection_area

    # Calcular o IoU
    iou = intersection_area / union_area

    return iou

# Exemplo de uso:

box2 = []  # Bounding box 2 no formato YOLO
box1 = []

file_1 = open("/content/output_object_0.txt", "r")
for values in file_1.read().split():
  box1.append(float(values))
print(box1[1:5])

file_2 = open("/content/CsiLab-BrainTumor-3/train/labels/45_jpg.rf.36ef270424c1c2114e972a731368c5df.txt", "r")
for values in file_2.read().split():
  box2.append(float(values))
print(box2[1:5])
iou = calculate_iou(box1[1:5], box2[1:5])
print("IoU:", iou)


[0.42410714285714285, 0.4375, 0.21875, 0.25892857142857145]
[0.42410714285714285, 0.5691964285714286, 0.5044642857142857, 0.5558035714285714]
IoU: 0.09001546989815644
